In this file, we will go through to a text classification model with TensorFlow on movie reviews as positive or negative using the text of the reviews. This is a binary classification problem, which is an important and widely applicable type of machine learning problem.

### Text Classification with TensorFlow

We’ll walk through the basic application of transfer learning with TensorFlow Hub and Keras. We will be using **IMDB** dataset which contains the text of **50,000 movie reviews** from the internet movie database. 

These are divided into 25,000 assessments for training and 25,000 assessments for testing. The training and test sets are balanced in a way that they contain an equal number of positive and negative reviews.

In [2]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [3]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


Although the dataset we are using here is available online to download, but we will simply load the data using TensorFlow. It means we don’t need to download the dataset from any external sources.

In [4]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dataset imdb_reviews downloaded and prepared to C:\Users\Waqas.Ali\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


### Data Exploration

Let’s have a look at the data to figure out what we are going to work with. we will simply print the first 10 samples from the dataset:

In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

Let’s print the first 10 labels from the data set:

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

### Building Text Classification Model

To build a model for the task of Text Classification with TensorFlow, we will use a pre-trained model provided by TensorFlow which is known by the name TensorFlow Hub. Let’s first create a Keras layer that uses a TensorFlow Hub model to the embed sentences, and try it out on some sample input:

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

Now build the model on the complete dataset:

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


### Compile The Model

Now, We will compile the model by using the loss function and the adam optimizer:

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Tran The Text Classification Model

Train the model for 20 epochs in mini-sets of 512 samples. These are 20 iterations on all the samples of the tensors `x_train` and `y_train`. During training, monitor model loss and accuracy on the 10,000 samples in the validation set:

In [12]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 8s 178ms/step - loss: 0.9307 - accuracy: 0.4512 - val_loss: 0.6892 - val_accuracy: 0.5664
Epoch 2/20
30/30 [==============================] - 4s 125ms/step - loss: 0.6547 - accuracy: 0.5893 - val_loss: 0.5799 - val_accuracy: 0.6805
Epoch 3/20
30/30 [==============================] - 4s 124ms/step - loss: 0.5612 - accuracy: 0.6896 - val_loss: 0.5256 - val_accuracy: 0.7199
Epoch 4/20
30/30 [==============================] - 4s 121ms/step - loss: 0.5057 - accuracy: 0.7356 - val_loss: 0.4882 - val_accuracy: 0.7459
Epoch 5/20
30/30 [==============================] - 4s 127ms/step - loss: 0.4704 - accuracy: 0.7673 - val_loss: 0.4593 - val_accuracy: 0.7844
Epoch 6/20
30/30 [==============================] - 4s 141ms/step - loss: 0.4320 - accuracy: 0.7982 - val_loss: 0.4319 - val_accuracy: 0.7911
Epoch 7/20
30/30 [==============================] - 4s 133ms/step - loss: 0.4023 - accuracy: 0.8142 - val_loss: 0.4101 - val_accuracy: 0.8014
Epoch 

#### Evaluating The Model

Let’s see how text classification model works. Two values will be returned. Loss and accuracy rate:

In [13]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3244 - accuracy: 0.8625
loss: 0.324
accuracy: 0.862


So our Text Classification Model achieved an accuracy rate of **85 percent** which is generally appreciated.